In [1]:
import numpy as np
import pandas as pd

# CRITIC Weight Calculation
def calculate_critic_weights(df, criteria_cols):
    normalized = df[criteria_cols].copy()
    for col in criteria_cols:
        normalized[col] = normalized[col] / np.sqrt((normalized[col] ** 2).sum())
    std_dev = normalized.std()
    correlation_matrix = normalized.corr()
    conflict = 1 - correlation_matrix.abs()
    conflict_sum = conflict.sum() - 1
    critic_weights = std_dev * conflict_sum
    critic_weights = critic_weights / critic_weights.sum()
    return critic_weights

# TOPSIS Function with CRITIC Weights
def topsis(df, criteria_cols, benefit_cols, cost_cols, user_row_index):
    # Step 1: Calculate CRITIC weights
    critic_weights = calculate_critic_weights(df, criteria_cols)

    # Step 2: Normalize the decision matrix
    decision_matrix = df[criteria_cols].values
    norm = np.sqrt((decision_matrix ** 2).sum(axis=0))
    normalized_matrix = decision_matrix / norm

    # Step 3: Create a weighted normalized decision matrix
    weighted_matrix = normalized_matrix * critic_weights.values

    # Step 4: Ensure user input is the ideal solution
    ideal = weighted_matrix[user_row_index]  # User input as ideal
    anti_ideal = []
    for i, col in enumerate(criteria_cols):
        if col in benefit_cols:
            anti_ideal.append(weighted_matrix[:, i].min()) # Ideal Worst for Benefit Criterion
        elif col in cost_cols:
            anti_ideal.append(weighted_matrix[:, i].max()) # Ideal Worst for Cost Criterion
        else:
            raise ValueError(f"Criterion '{col}' must be either benefit or cost.")
    anti_ideal = np.array(anti_ideal)

    # Step 5: Calculate the distance to the ideal best and worst
    distance_to_ideal = np.sqrt(((weighted_matrix - ideal) ** 2).sum(axis=1))
    distance_to_anti_ideal = np.sqrt(((weighted_matrix - anti_ideal) ** 2).sum(axis=1))

    # Step 6: Calculate the relative closeness to the ideal solution
    relative_closeness = distance_to_anti_ideal / (distance_to_ideal + distance_to_anti_ideal)

    # Step 7: Add the TOPSIS score and rank to the DataFrame
    df['TOPSIS_Score'] = relative_closeness
    df['Rank'] = df['TOPSIS_Score'].rank(ascending=False)

    return df, critic_weights

# User Input and Data Preparation
personal_factors_data = {
    "Risk Level": [1, 2, 3, 4, 5],
    "Monthly SIP Amount": [1000, 5000, 10000, 25000, 50000],
    "Age": [60, 45, 30, 25, 20],
    "Time Horizon": [1, 3, 5, 10, 20],
    "Debt Level as % of Income": [50,40,30,20,10],
}

In [2]:
import pandas as pd
import numpy as np

# Load Data
file_path = "final_ratios1.csv"
df = pd.read_csv(file_path)

# Define Criteria
cost_criteria = ["Sharpe", "Treynor"]  
benefit_criteria = ["Drawdown", "StdDev", "Beta", "HHI"]  

# Normalize Data
def normalize_matrix(matrix, benefit_criteria):
    # Step 1: Shift the data to ensure all values are positive
    matrix_shifted = matrix.copy()
    for col in matrix.columns:
        shift_value = abs(matrix[col].min())  # Absolute value of the smallest element in the column
        matrix_shifted[col] = matrix[col] + shift_value  # Shift all values in the column
    
    # Step 2: Normalize the shifted data
    norm_matrix = matrix_shifted.copy()
    for col in matrix_shifted.columns:
        if col in benefit_criteria:  # Benefit criteria
            norm_matrix[col] = matrix_shifted[col] / matrix_shifted[col].sum()
        else:  # Cost criteria
            norm_matrix[col] = matrix_shifted[col].min() / matrix_shifted[col]
    
    return norm_matrix

# Normalize the dataset
normalized_matrix = normalize_matrix(df[benefit_criteria + cost_criteria], benefit_criteria)

# Handle Zeros for Logarithmic Calculation
normalized_matrix.replace(0, 1e-10, inplace=True)

# Calculate Entropy
def calculate_entropy(norm_matrix):
    k = 1 / np.log(len(norm_matrix))  # Constant k
    entropy = -k * (norm_matrix * np.log(norm_matrix)).sum()
    diversification = 1 - entropy
    weights = diversification / diversification.sum()
    return entropy, weights
    

entropy, weights = calculate_entropy(normalized_matrix)

# Calculate Diversification and Weights
diversification = 1 - entropy
weights = diversification / diversification.sum()

In [3]:
def normalize_matrix(matrix, benefit_criteria):
    # Step 1: Shift the data to ensure all values are positive
    matrix_shifted = matrix.copy()
    for col in matrix.columns:
        shift_value = abs(matrix[col].min())  # Absolute value of the smallest element in the column
        matrix_shifted[col] = matrix[col] + shift_value  # Shift all values in the column
    
    # Step 2: Normalize the shifted data
    norm_matrix = matrix_shifted.copy()
    for col in matrix_shifted.columns:
        if col in benefit_criteria:  # Benefit criteria
            norm_matrix[col] = matrix_shifted[col] / matrix_shifted[col].sum()
        else:  # Cost criteria
            norm_matrix[col] = matrix_shifted[col].min() / matrix_shifted[col]
    
    return norm_matrix

def copras(df, benefit_criteria,weights):
    # Extract scheme codes and names from the DataFrame
    scheme_codes = df['Code']
    scheme_names = df['Name']
    
    # Remove non-numeric columns (i.e., 'Scheme Code' and 'Scheme Name')
    matrix = df.drop(columns=['Code', 'Name'])
    
    # Step 1: Normalize the matrix
    norm_matrix = normalize_matrix(matrix, benefit_criteria)

    # Step 2: Assign equal weights to all criteria (or set your own)
    num_criteria = len(matrix.columns)

    # Step 3: Calculate weighted normalized decision matrix
    weighted_norm_matrix = norm_matrix * weights

    # Step 4: Calculate positive and negative deviations
    # Positive deviations (S+): sum of weighted normalized values for each alternative
    S_plus = weighted_norm_matrix.max(axis=0) - weighted_norm_matrix
    S_plus_sum = S_plus.sum(axis=1)  # Sum of positive deviations for each alternative
    
    # Negative deviations (S-): sum of weighted normalized values for each alternative
    S_minus = weighted_norm_matrix - weighted_norm_matrix.min(axis=0)
    S_minus_sum = S_minus.sum(axis=1)  # Sum of negative deviations for each alternative

    # Step 5: Calculate COPRAS performance index (P)
    P = S_plus_sum / (S_plus_sum + S_minus_sum)

    # Step 6: Rank the alternatives based on the performance index
    rankings = P.sort_values(ascending=False).index

    # Step 7: Attach scheme codes and names to the results
    result = pd.DataFrame({
        'Code': scheme_codes[rankings],
        'Scheme Name': scheme_names[rankings],
        'Performance Index': P[rankings].values
    })
    
    # Return the performance indices and rankings
    return result

# Define which criteria are benefit and which are cost
benefit_criteria = ["Drawdown", "StdDev", "Beta", "HHI"] 

# Run COPRAS method
result = copras(df, benefit_criteria,weights=weights)


In [4]:
result['Risk_Category'] = pd.qcut(result['Performance Index'], q=5, labels=[1, 2, 3, 4, 5])

In [5]:
from mftool import Mftool
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf

# Initialize mftool
mf = Mftool()


# Define functions to calculate CAGR and Jensen's Alpha
def calculate_cagr(start_nav, end_nav, years):
   
    return ((end_nav / start_nav) ** (1 / years) - 1) * 100

def calculate_jensens_alpha(returns, benchmark_returns, beta, risk_free_rate=0.06):
    
    expected_returns = risk_free_rate + beta * (benchmark_returns - risk_free_rate)
    return returns - expected_returns

# Define date ranges for 3-year and 5-year periods
today = datetime.today()
three_years_ago = today - timedelta(days=3 * 365)
five_years_ago = today - timedelta(days=5 * 365)

# Fetch NIFTY 50 data using yfinance
nifty_data = yf.download("^NSEI", 
                         start=five_years_ago.strftime('%Y-%m-%d'), 
                         end=today.strftime('%Y-%m-%d'))

# If yfinance returns a multi-index DataFrame, reset the column index
if isinstance(nifty_data.columns, pd.MultiIndex):
    nifty_data = nifty_data['Adj Close']  # Select only the 'Adj Close' column
else:
    nifty_data = nifty_data[['Adj Close']]  # Regular single-index case


# Check the DataFrame to confirm it's correctly formatted
print(nifty_data.head())

# Calculate NIFTY benchmark returns for 3 and 5 years
nifty_5yr_start = nifty_data.iloc[0]['^NSEI']
nifty_3yr_start = nifty_data[nifty_data.index >= three_years_ago.strftime('%Y-%m-%d')].iloc[0]['^NSEI']
nifty_end = nifty_data.iloc[-1]['^NSEI']

benchmark_3yr_return = (nifty_end / nifty_3yr_start - 1) * 100
benchmark_5yr_return = (nifty_end / nifty_5yr_start - 1) * 100
# Initialize new columns
result['3_Year_CAGR'] = np.nan
result['5_Year_CAGR'] = np.nan
result['Jensens_Alpha'] = np.nan


# Loop through each mutual fund scheme and calculate metrics
for index, row in result.iterrows():
    scheme_code = str(row['Code'])

    try:
        # Fetch historical NAV data for the mutual fund scheme
        nav_data = mf.get_scheme_historical_nav(scheme_code, as_Dataframe=True)

        if nav_data.empty:
            print(f"No NAV data found for Scheme Code {scheme_code}.")
            continue

        print(f"\nProcessing Scheme Code: {scheme_code}")
        print("NAV Data Head:")
        print(nav_data.head())  # Display the first few rows to debug

        # Reset index to access 'date' as a column
        nav_data = nav_data.reset_index()
        nav_data['nav'] = nav_data['nav'].astype(float)  # Ensure NAV is float
        nav_data['date'] = pd.to_datetime(nav_data['date'], format='%d-%m-%Y')  # Ensure 'date' is datetime

        print("NAV Data After Reset:")
        print(nav_data.head())  # Debugging reset DataFrame

        # Filter rows for 3-year and 5-year periods
        three_years_ago_nav = nav_data[nav_data['date'] <= three_years_ago]
        five_years_ago_nav = nav_data[nav_data['date'] <= five_years_ago]

        # Use first matching row (latest NAV on or before the target date)
        start_nav_3yr = three_years_ago_nav.iloc[0]['nav'] if not three_years_ago_nav.empty else np.nan
        start_nav_5yr = five_years_ago_nav.iloc[0]['nav'] if not five_years_ago_nav.empty else np.nan

        # Use the latest NAV (first row in the original data)
        end_nav = nav_data.iloc[0]['nav']
        
        print(f"3-Year Start NAV: {start_nav_3yr}")
        print(f"5-Year Start NAV: {start_nav_5yr}")
        print(f"End NAV: {end_nav}")

        # Calculate CAGR values
        result.loc[index, '3_Year_CAGR'] = calculate_cagr(start_nav_3yr, end_nav, 3)
        result.loc[index, '5_Year_CAGR'] = calculate_cagr(start_nav_5yr, end_nav, 5)

    except Exception as e:
        print(f"Error processing Scheme Code {scheme_code}: {e}")



[*********************100%***********************]  1 of 1 completed


Ticker             ^NSEI
Date                    
2020-01-24  12248.250000
2020-01-27  12119.000000
2020-01-28  12055.799805
2020-01-29  12129.500000
2020-01-30  12035.799805

Processing Scheme Code: 147559
NAV Data Head:
                 nav  dayChange
date                           
23-08-2023  12.92400     0.0022
22-08-2023  12.92180     0.0022
21-08-2023  12.91960     0.0023
20-08-2023  12.91730     0.0043
18-08-2023  12.91300     0.0022
NAV Data After Reset:
        date      nav  dayChange
0 2023-08-23  12.9240     0.0022
1 2023-08-22  12.9218     0.0022
2 2023-08-21  12.9196     0.0023
3 2023-08-20  12.9173     0.0043
4 2023-08-18  12.9130     0.0022
3-Year Start NAV: 11.9197
5-Year Start NAV: 10.4916
End NAV: 12.924

Processing Scheme Code: 147560
NAV Data Head:
                 nav  dayChange
date                           
23-08-2023  13.05580     0.0024
22-08-2023  13.05340     0.0023
21-08-2023  13.05110     0.0024
20-08-2023  13.04870     0.0045
18-08-2023  13.04420     0.

In [6]:
result.drop(columns=['Jensens_Alpha'],inplace=True)

In [7]:
result.dropna(subset=['3_Year_CAGR'],inplace=True)

In [8]:
result.to_csv("results.csv")

In [9]:
def get_basket(df_personal_factors,result):
    risk_basket1 = result.loc[result['Risk_Category']==5] #low copras score  = higher risk
    risk_basket2 = result.loc[result['Risk_Category']==4] 
    risk_basket3 = result.loc[result['Risk_Category']==3]
    risk_basket4 = result.loc[result['Risk_Category']==2]
    risk_basket5 = result.loc[result['Risk_Category']==1] 
    risk_level = df_personal_factors.loc[df_personal_factors['Rank'] == 2, 'Risk Level'].iloc[0]
    if risk_level == 1:
        return risk_basket1.head().sort_values(by='3_Year_CAGR',ascending=False)
    elif risk_level == 2:
        return risk_basket2.head().sort_values(by='3_Year_CAGR',ascending=False)
    elif risk_level == 3:
        return risk_basket3.head().sort_values(by='3_Year_CAGR',ascending=False)
    elif risk_level == 4:
        return risk_basket4.head().sort_values(by='3_Year_CAGR',ascending=False)
    else:
        return risk_basket5.head().sort_values(by='3_Year_CAGR',ascending=False)

In [24]:
print("\nPlease enter values for your personal factors:")
user_row = {}
for factor in personal_factors_data.keys():
    user_row[factor] = float(input(f"Enter value for {factor}: "))
for factor in personal_factors_data.keys():
    personal_factors_data[factor].append(user_row[factor])

df_personal_factors = pd.DataFrame(personal_factors_data)

criteria_cols = ["Monthly SIP Amount", "Age", "Time Horizon", "Debt Level as % of Income"]
benefit_cols = ["Monthly SIP Amount", "Time Horizon"]
cost_cols = ["Age", "Debt Level as % of Income"]

# Run TOPSIS, treating the user input as the ideal solution (last row)
user_row_index = len(df_personal_factors) - 1



Please enter values for your personal factors:


Enter value for Risk Level:  5
Enter value for Monthly SIP Amount:  100000
Enter value for Age:  39
Enter value for Time Horizon:  1
Enter value for Debt Level as % of Income:  10


In [25]:
df_personal_factors_topsis, critic_weights = topsis(df_personal_factors, criteria_cols, benefit_cols, cost_cols, user_row_index)

In [26]:
get_basket(df_personal_factors_topsis,result)

,Code,Scheme Name,Performance Index,Risk_Category,3_Year_CAGR,5_Year_CAGR
2041,120826,quant Large & Mid Cap Fund - Growth Option-Dir...,0.939550,1,21.712769,25.401838
2368,120665,UTI Large & Mid Cap Fund - Direct Plan - Growt...,0.939648,1,19.823771,21.849766
2843,120494,JM ELSS Tax Saver Fund (Direct) - Growth Option,0.939714,1,17.523706,20.462766
164,148745,Kotak Nifty Next 50 Index Fund - Direct Plan -...,0.939698,1,15.290854,NaN
2511,102394,UTI Mid Cap Fund-Growth Option,0.939672,1,14.324875,21.639475


In [20]:
result

,Code,Scheme Name,Performance Index,Risk_Category,3_Year_CAGR,5_Year_CAGR
2540,147559,SBI Fixed Maturity Plan (FMP) - Series 14 (384...,0.993129,5,2.733135,4.258397
2389,147560,SBI Fixed Maturity Plan (FMP) - Series 11 (117...,0.993127,5,2.867886,4.444511
435,113142,Navi Conservative Hybrid Fund-Growth,0.993048,5,5.501561,5.433857
410,119156,Navi Conservative Hybrid Fund-Direct Plan-Grow...,0.993025,5,7.158588,7.160183
3066,146173,SBI Debt Fund Series C - 38 (389 Days) - Direc...,0.993004,5,2.515906,3.997126
...,...,...,...,...,...,...
330,139337,Nippon India Capital Protection Oriented Fund ...,0.589681,1,0.000000,0.000000
331,139335,Nippon India Dual Advantage Fixed Tenure Fund ...,0.589681,1,0.000000,0.000000
208,145669,Nippon India Interval Fund V - Series 2 - Dire...,0.500346,1,0.000000,0.000000
225,145670,Nippon India Interval Fund Annual Interval Fun...,0.500344,1,0.000000,0.000000
